In [ ]:
from urllib.request import Request, urlopen
import urllib
from bs4 import BeautifulSoup
from time import sleep
import csv
import re

In [ ]:
# url to scrape the songs list from
#base_url = "http://www.hindilyrics.net/lyrics/of-{}.html"

eng_url = "http://www.hindilyrics.net/lyrics/of-{}.html"

hindi_url = "https://www.hinditracks.in/{}-lyrics"

songs = ["Mere Naam Tu","Bhankas","bekhayali","Aira Gaira","Paisa",
         "Jugraafiya","Issaqbaazi","Husn Parcham",
        "Dil Royi Jaye","Mukhda Vekh ke","Chale Aana","Humne Rait pe","Hume Tumse Pyaar Kitna",
         "Manmohini","Phir Mulaaqat"]

In [ ]:
hindi_lyrics = []
english_lyrics = []
for song in songs:
    eng_ly = []
    song_ = '%20'.join(x for x in song.lower().split())
    #extract English Lyrics
    req = eng_url.format(song_)
    artist_url = Request(req, headers={'User-Agent': 'Mozilla/5.0'})
    
    print("Going to url : ", req)
    try:
        html_page = urlopen(artist_url).read()
    except urllib.error.HTTPError as e:
        print('Lyrics Not Found')
        continue
    try:
        soup = BeautifulSoup(html_page, 'html.parser')
        data = soup.find('pre').text.strip()
        for lines in data.splitlines():
            eng_ly.append(lines.strip())
            #print(lines.strip())
    except AttributeError as a:
        print('Lyrics Not Found')
    #print(eng_ly)
        
    print('------------------------------------------------')

    #extract Hindi Lyrics
    hin_ly = []
    song_ = '-'.join(x for x in song.lower().split())
    req = hindi_url.format(song_)
    artist_url = Request(req, headers={'User-Agent': 'Mozilla/5.0'})
    
    print("Going to url : ", req)
    try:
        html_page = urlopen(artist_url).read()
    except urllib.error.HTTPError as e:
        print('Lyrics Not Found')
        continue

    try:
        soup = BeautifulSoup(html_page, 'html.parser')
        end = False
        divs = soup.find('div',attrs={'id':'hindilyrics'})
        hin_ly.append(divs.text.strip())
        #print(divs.text.strip())
        p = divs.find_next('p')
        hin_ly.append(p.text.strip())
        #print(p.text.strip())
        while not end:
            p = p.find_next('p')
            #print(p.text.strip())
            hin_ly.append(p.text.strip())
            if re.match('[A-Za-z0-9]',p.find_next('p').text):
                #print(hin_ly)
                end = True
    except AttributeError as error:
        print('Lyrics Not Found')
    if len(eng_ly)!=0 and len(hin_ly)!=0:
        hindi_lyrics.append(hin_ly)
        english_lyrics.append(eng_ly)
    else:
        continue
    print('------------------------------------------------')

print(len(hindi_lyrics))
print(len(english_lyrics))
    
    

In [ ]:
file = open('./hindi_lyrics.txt','w')
for i in range(0,len(hindi_lyrics)):
    ly = ''
    lines = hindi_lyrics[i]
    for l in lines:
        if not l.startswith('#'):
            l = re.sub(r'[a-zA-Z0-9\.\:\,\(\)]','',l)
            file.write(l)
file.close()

In [ ]:
file = open('./english_lyrics.txt','w')
for i in range(0,len(english_lyrics)):
    for lines in english_lyrics[i]:
        lines = re.sub(r'[\.\,\(\)]','',lines.lower())
        file.writelines(lines+'\n')
file.close()

In [ ]:
hindi_lyrics = '/home/stuti/Documents/Jan_Intern/GPU/GPU_Train_transliteration/train/hindi_lyrics.txt'
english_lyrics = '/home/stuti/Documents/Jan_Intern/GPU/GPU_Train_transliteration/train/english_lyrics.txt'
with open(hindi_lyrics) as file:
    txt = file.read()
    hindi_words = txt.split()
with open(english_lyrics) as file:
    txt = file.read()
    english_words = txt.split()

In [ ]:
import csv
with open('./words_from_songs.csv','w') as outfile:
    writer = csv.writer(outfile,escapechar=' ',quoting=csv.QUOTE_NONE)
    writer.writerow(['English','Hindi'])
    for i in range(len(hindi_words)):
        writer.writerow([english_words[i],hindi_words[i]])